In [25]:
import json
import os

ADDRESS = "0xe6d48bf4ee912235398b96e16db6f310c21e82cb"
PATH = "./raw_transactions/%s" % (ADDRESS)


def load_files(files):
    print("Loading transactions...")
    transactions = []
    for file in files:
        print("Loading %s..." % (file))
        if file == ".DS_Store":
            continue

        with open(PATH + "/" + file) as f:
            transactions += json.loads(f.read()).get("items")
    return transactions


files = os.listdir(PATH)
transactions = load_files(files)


# with open("raw_transactions/0xb/{}.json".format(ADDRESS), 'rb') as f:
#     transactions = json.loads(f.read()).get("items")

Loading transactions...
Loading 0xe6d48bf4ee912235398b96e16db6f310c21e82cb_34_500.json...
Loading 0xe6d48bf4ee912235398b96e16db6f310c21e82cb_35_500.json...
Loading 0xe6d48bf4ee912235398b96e16db6f310c21e82cb_36_114.json...
Loading 0xe6d48bf4ee912235398b96e16db6f310c21e82cb_23_500.json...
Loading 0xe6d48bf4ee912235398b96e16db6f310c21e82cb_22_500.json...
Loading 0xe6d48bf4ee912235398b96e16db6f310c21e82cb_29_500.json...
Loading 0xe6d48bf4ee912235398b96e16db6f310c21e82cb_28_500.json...
Loading 0xe6d48bf4ee912235398b96e16db6f310c21e82cb_1_500.json...
Loading 0xe6d48bf4ee912235398b96e16db6f310c21e82cb_0_500.json...
Loading 0xe6d48bf4ee912235398b96e16db6f310c21e82cb_17_500.json...
Loading 0xe6d48bf4ee912235398b96e16db6f310c21e82cb_16_500.json...
Loading 0xe6d48bf4ee912235398b96e16db6f310c21e82cb_24_500.json...
Loading 0xe6d48bf4ee912235398b96e16db6f310c21e82cb_25_500.json...
Loading 0xe6d48bf4ee912235398b96e16db6f310c21e82cb_6_500.json...
Loading 0xe6d48bf4ee912235398b96e16db6f310c21e82cb_7_50

In [26]:
from blockchain.domains.eth_transaction import EthTransaction

eth_transactions = list(map(lambda transaction: EthTransaction(transaction), transactions))
eth_transactions = list(filter(lambda eth_transactions: eth_transactions.successful == True, eth_transactions))
print(len(eth_transactions))

16653


In [27]:
eth_log_events = list(map(
    lambda cleaned_transaction: cleaned_transaction.log_events, eth_transactions))
eth_log_events = sum(eth_log_events, [])
print(len(eth_log_events))

33358


In [28]:
from blockchain.services.eth_token_transfer_extractor import EthTokenTransferExtractor
from blockchain.services.eth_token_sale_extractor import EthTokenSaleExtractor

eth_token_transfer_extractor = EthTokenTransferExtractor()
eth_token_sale_extractor = EthTokenSaleExtractor()
eth_token_transfer_list = list()
eth_token_non_transfer_list = list()
eth_log_event_list = list()

for eth_log_event in eth_log_events:
    eth_log_event_list.append(eth_log_event.to_json())
    eth_token_transfer = eth_token_transfer_extractor.extract_token_transfer_from_log_event(eth_log_event)
    if eth_token_transfer != None:
        eth_token_transfer_list.append(eth_token_transfer.to_json())
    else:
        eth_token_non_transfer = eth_token_sale_extractor.extract_token_non_transfer_from_log_event(eth_log_event)
        eth_token_non_transfer_list.append(eth_token_non_transfer.to_json()) if eth_token_non_transfer != None else None
    
sales_df = eth_token_sale_extractor.extract_token_sales_from_log_events(eth_log_event_list, eth_token_non_transfer_list)

eth_token_transfer_df = eth_token_transfer_extractor.reconnect_transfer_relationship(eth_token_transfer_list)

eth_token_transfer_df = eth_token_transfer_df.sort_values(['log_index'], ascending=False).groupby(
            ['token_id', 'transaction_hash']).head(1)

eth_token_transfer_df.to_csv("a.csv")
final_df = eth_token_transfer_df.merge(sales_df, left_on=['from_address', 'to_address', 'transaction_hash'], right_on=['maker', 'taker', 'transaction_hash'], how='left')


In [29]:
# a = filter(lambda log_event: log_event.transaction_hash == "0x6833305ac95da67f7fc515116145da311745c5b2b70b9c20b2e22a0a20113078",eth_log_events)
# print(eth_token_transfer_extractor.extract_token_transfer_from_log_event(list(a)[0]))
# print(final_df[final_df['action'] == "Minted"])
# print(final_df[final_df['transaction_hash'] == "0x6833305ac95da67f7fc515116145da311745c5b2b70b9c20b2e22a0a20113078"])

In [30]:
import gzip

with gzip.open("output/{}".format(ADDRESS + ".json.gz"), 'w') as f:
    f.write(final_df.to_json(orient="records").encode('utf-8'))

In [31]:
final_df.to_csv("final.csv")